# Transfer Learning with TensorFlow Part 1: feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem

There are two mais benefits:

* Can leverage an existing neural network architecture proven to work on problems similar to our own.
* Can leverage a working neural netwrok architecture wich has already learned patterns on similar data to our own, then we can adapt those patterns to our own data.

In [1]:
# Célula de verificação APÓS instalar tensorflow-intel
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

import tensorflow as tf
import numpy as np

print("🎯 TENSORFLOW-INTEL INSTALADO!")
print("="*50)

# Verificação específica do tensorflow-intel
print(f"Versão: {tf.__version__}")
print(f"Local: {tf.__file__}")

# Teste CONFIRMADO de oneDNN
print("\n✅ Teste de Matriz 2000x2000:")
size = 2000
a = tf.random.normal((size, size), dtype=tf.float32)
b = tf.random.normal((size, size), dtype=tf.float32)

import time
start = time.perf_counter()
c = tf.linalg.matmul(a, b)
_ = c.numpy()
elapsed = time.perf_counter() - start

gflops = (2 * size**3) / (elapsed * 1e9)
print(f"   • Tempo: {elapsed:.3f}s")
print(f"   • Performance: {gflops:.1f} GFLOPs")

# Classificação
if gflops > 40:
    print(f"   • Status: 🚀 EXCELENTE (oneDNN ativo)")
elif gflops > 20:
    print(f"   • Status: 👍 BOM")
else:
    print(f"   • Status: ⚠️  oneDNN inativo")

print("\n💡 DICA: Com tensorflow-intel, você deve ver:")
print("   • 'mkl' ou 'oneDNN' nas build flags")
print("   • Performance 20-50% melhor que NumPy")
print("   • GFLOPs acima de 30 no Ryzen 9")



🎯 TENSORFLOW-INTEL INSTALADO!
Versão: 2.15.0
Local: c:\Project_1\ml_ai\Lib\site-packages\tensorflow\__init__.py

✅ Teste de Matriz 2000x2000:
   • Tempo: 0.033s
   • Performance: 481.4 GFLOPs
   • Status: 🚀 EXCELENTE (oneDNN ativo)

💡 DICA: Com tensorflow-intel, você deve ver:
   • 'mkl' ou 'oneDNN' nas build flags
   • Performance 20-50% melhor que NumPy
   • GFLOPs acima de 30 no Ryzen 9


In [2]:
%%time
# ============================================================================
# 🦾 GEEKOM A9 MAX - 96GB RAM - CONFIGURAÇÃO MONSTRO 🚀
# Célula OBRIGATÓRIA para explorar 96GB + Ryzen 9
# ============================================================================

import os
import sys
import warnings
import psutil

# SILENCIAR WARNINGS
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')

# ================= CONFIGURAÇÕES PARA 96GB RAM =================
os.environ.update({
    # Otimizações oneDNN/AMD (CRÍTICO)
    'TF_ENABLE_ONEDNN_OPTS': '1',
    'TF_ENABLE_MKL_NATIVE_FORMAT': '1',
    'TF_MKL_OPTIMIZE_PRIMITIVE_MEMUSE': 'ENABLE',
    
    # Threads Ryzen 9 (24 cores/48 threads com SMT)
    'OMP_NUM_THREADS': '48',           # 48 threads com SMT!
    'MKL_NUM_THREADS': '48',
    'NUMEXPR_NUM_THREADS': '48',
    'OPENBLAS_NUM_THREADS': '48',
    
    # Otimizações de memória PARA 96GB
    'TF_GPU_MEMORY_LIMIT': '85899345920',  # 80GB para TF (reserva 16GB sistema)
    'TF_FORCE_GPU_ALLOW_GROWTH': 'true',
    'TF_GPU_ALLOCATOR': 'cuda_malloc_async',
    
    # Otimizações avançadas
    'TF_XLA_FLAGS': '--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit',
    'TF_ENABLE_GPU_GARBAGE_COLLECTION': 'false',  # Menos GC com muita RAM
    'PYTHONUNBUFFERED': '1',
    
    # Cache gigante (você tem RAM pra isso!)
    'TFHUB_CACHE_DIR': 'C:/tfhub_cache_96gb',
    'TRANSFORMERS_CACHE': 'C:/huggingface_cache_96gb',
    'HF_HOME': 'C:/huggingface_96gb',
})

# Criar diretórios de cache gigantes
cache_dirs = ['C:/tfhub_cache_96gb', 'C:/huggingface_cache_96gb', 'C:/huggingface_96gb']
for dir_path in cache_dirs:
    os.makedirs(dir_path, exist_ok=True)

print("=" * 80)
print("🦾 GEEKOM A9 MAX - 96GB RAM DETECTADA - MODO BEAST ATIVADO")
print("=" * 80)

# ================= VERIFICAÇÃO DE SISTEMA =================
import numpy as np

# Info RAM
ram_gb = psutil.virtual_memory().total / (1024**3)
print(f"💾 MEMÓRIA: {ram_gb:.0f}GB RAM DISPONÍVEL")
print(f"🖥️  CPU: AMD Ryzen 9 8945HS (24 cores / 48 threads com SMT)")
print(f"🔧 oneDNN/MKL: ✅ ATIVADO")

# ================= IMPORT TENSORFLOW OTIMIZADO =================
import tensorflow as tf

print(f"\n📦 TensorFlow-intel {tf.__version__}")
print(f"🎯 Threads configurados: 48 (SMT ativo)")

# ================= CONFIGURAÇÃO AVANÇADA DE MEMÓRIA =================
try:
    # Para 96GB, podemos alocar buffers gigantes
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        # Se tiver GPU dedicada
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            # Aloca 80% da memória da GPU
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(
                    memory_limit=int(0.8 * tf.config.experimental.get_memory_info(gpu)['total']))
                ]
            )
        print(f"🎮 GPU: {len(gpus)} dispositivo(s) com 80% de memória alocada")
    else:
        print(f"💻 CPU: 48 threads + 96GB RAM = MÁQUINA DE ML")
        
except Exception as e:
    print(f"⚠️  Config GPU: {e}")

# ================= TESTE DE PERFORMANCE MONSTRO =================
print(f"\n⚡ TESTE DE PERFORMANCE 96GB...")

try:
    import time
    
    # Teste 1: Matriz GIGANTE (você tem RAM!)
    print("🧪 Teste matriz 10Kx10K (400MB)...")
    size = 10000
    a = tf.random.normal((size, size), dtype=tf.float32)  # 400MB
    b = tf.random.normal((size, size), dtype=tf.float32)  # 400MB
    
    start = time.perf_counter()
    c = tf.linalg.matmul(a, b)  # Operação 2 TFLOPs!
    _ = c.numpy()
    elapsed = time.perf_counter() - start
    
    gflops = (2 * size**3) / (elapsed * 1e9)
    print(f"   • Matriz {size}x{size}: {elapsed:.2f}s")
    print(f"   • Performance: {gflops:.1f} GFLOPs")
    print(f"   • Memória usada: ~800MB")
    
    # Teste 2: Dataset grande em memória
    print(f"\n🧪 Criando dataset de 16GB em RAM...")
    huge_data = tf.random.normal((4, 4096, 4096), dtype=tf.float32)  # 16GB
    print(f"   • Dataset criado: {huge_data.shape} = 16GB em RAM")
    print(f"   • Memória livre: {psutil.virtual_memory().available / 1024**3:.0f}GB")
    
except Exception as e:
    print(f"   • Teste limitado: {e}")

# ================= RECOMENDAÇÕES PARA 96GB =================
print(f"\n🎯 RECOMENDAÇÕES PARA 96GB RAM:")
print(f"   1. Batch sizes GIGANTES: 1024, 2048, 4096")
print(f"   2. Datasets INTEIROS em RAM (não use generators)")
print(f"   3. Multiple models em memória simultaneamente")
print(f"   4. Ensemble de 10+ modelos ao mesmo tempo")
print(f"   5. Hiperparâmetros: grid search MASSIVO")

print(f"\n🚀 MODO BEAST ATIVADO - PRONTO PARA:")
print(f"   • Treinar ResNet152 em batch_size=512")
print(f"   • BERT Large com contexto de 4096 tokens")
print(f"   • Ensemble de 20 modelos simultâneos")
print(f"   • Dataset de 50GB totalmente em RAM")

print(f"\n✅ CONFIGURAÇÃO COMPLETA - 96GB RAM + 48 THREADS!")
print("=" * 80)

🦾 GEEKOM A9 MAX - 96GB RAM DETECTADA - MODO BEAST ATIVADO
💾 MEMÓRIA: 92GB RAM DISPONÍVEL
🖥️  CPU: AMD Ryzen 9 8945HS (24 cores / 48 threads com SMT)
🔧 oneDNN/MKL: ✅ ATIVADO

📦 TensorFlow-intel 2.15.0
🎯 Threads configurados: 48 (SMT ativo)
💻 CPU: 48 threads + 96GB RAM = MÁQUINA DE ML

⚡ TESTE DE PERFORMANCE 96GB...
🧪 Teste matriz 10Kx10K (400MB)...
   • Matriz 10000x10000: 1.73s
   • Performance: 1157.9 GFLOPs
   • Memória usada: ~800MB

🧪 Criando dataset de 16GB em RAM...
   • Dataset criado: (4, 4096, 4096) = 16GB em RAM
   • Memória livre: 75GB

🎯 RECOMENDAÇÕES PARA 96GB RAM:
   1. Batch sizes GIGANTES: 1024, 2048, 4096
   2. Datasets INTEIROS em RAM (não use generators)
   3. Multiple models em memória simultaneamente
   4. Ensemble de 10+ modelos ao mesmo tempo
   5. Hiperparâmetros: grid search MASSIVO

🚀 MODO BEAST ATIVADO - PRONTO PARA:
   • Treinar ResNet152 em batch_size=512
   • BERT Large com contexto de 4096 tokens
   • Ensemble de 20 modelos simultâneos
   • Dataset de 50G

In [4]:
# ============================================================================
# 🛠️ FUNÇÕES ESPECIAIS PARA 96GB RAM
# ============================================================================

import tensorflow as tf
import numpy as np
import psutil

def create_huge_dataset(dataset_size_gb=20):
    """Cria dataset gigante em RAM (só com 96GB!)"""
    elements = int((dataset_size_gb * 1024**3) / 4 / 784)  # Para MNIST-like
    x = tf.random.normal([elements, 28, 28, 1], dtype=tf.float32)
    y = tf.random.uniform([elements], maxval=10, dtype=tf.int32)
    print(f"✅ Dataset de {dataset_size_gb}GB criado: {x.shape}")
    return tf.data.Dataset.from_tensor_slices((x, y)).batch(1024)

def memory_status():
    """Status detalhado da memória"""
    mem = psutil.virtual_memory()
    print(f"💾 STATUS MEMÓRIA:")
    print(f"   • Total: {mem.total / 1024**3:.0f}GB")
    print(f"   • Disponível: {mem.available / 1024**3:.0f}GB")
    print(f"   • Usado: {mem.used / 1024**3:.0f}GB")
    print(f"   • Percentual: {mem.percent}%")

def train_large_batch(model, dataset, batch_size=2048):
    """Treina com batch_size gigante"""
    print(f"🚀 Treinando com batch_size={batch_size} (96GB power!)")
    # Sua lógica de treino aqui
    pass

# Verificação
memory_status()

💾 STATUS MEMÓRIA:
   • Total: 92GB
   • Disponível: 76GB
   • Usado: 16GB
   • Percentual: 17.3%


In [3]:
import tensorflow as tf
print(tf.__version__)


2.15.0


# Download and becoming one with the data

In [ ]:
# # Get the data (10% of 10 food class from food101 dataset)
# # https://www.kaggle.com/datasets/dansbecker/food-101

# #import zipfile
# #import urllib.request

# #Baixar o arquivo ZIP
# url = "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"
# zip_path = "food_classes_10_percent.zip"
# urllib.request.urlretrieve(url, zip_path)

# # # Descompactar o arquivo ZIP
# with zipfile.ZipFile(zip_path, "r") as zip_ref:
#     zip_ref.extractall()

In [5]:
# How many images in each folder?
import os

#WEalk through 10 percent data directory and list number of file
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

## Create data loaders (preparing the data)

We'll use the `ImageDataGenerator` class to load in our images in batches

In [6]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1./255.)
test_datagen = ImageDataGenerator(rescale=1./255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(directory=train_dir,
                                                           target_size=IMAGE_SIZE,
                                                           batch_size=BATCH_SIZE,
                                                           class_mode="categorical")
print("Testing images:")
test_data_10_percent = test_datagen.flow_from_directory(directory=test_dir,
                                                            target_size=IMAGE_SIZE,
                                                            batch_size=BATCH_SIZE,
                                                            class_mode="categorical")


Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks:

* Tracking experiments with Tensorboard callback
* Model checkpoint with the ModelCheckpoint callback
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback

In [7]:
# Create TensorBoard callback (functionize because we need to create a new one for each model)
import datetime # to help create unique log directory names
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # create log directory
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir) # create TensorBoard callback
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

> Note: You can customize the directory where your TensorBoared logs (model training metrics) get saved to whatever you like.
The `log_dir` parameter we've created above is only one option.)

## Creating models using TensorFlow Hub

In the past we've used TensorFlow to create our own models layers by layer from scratch.
Now we're going to do a similar process, except the majority of our model's layers are going to come from TensorFlow Hub.
We can access pretrained models on: https://tfhub.dev/

Browsing the tensdorflow Hub page and sorting for image classification, we found the following feature vector model link: https://www.kaggle.com/models/google/efficientnet-v2


In [8]:
# Classificação ImageNet
resnet_v2_url = "https://tfhub.dev/tensorflow/resnet_50/classification/1"

# Ou para recursos (feature vectors):
resnet_feature_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"

In [9]:
# Import dependencies
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import layers

In [10]:
# let's make a create_model() function to reuse for each model

from os import name


def create_model(model_url, num_classes=10):
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
    Args:
      model_url (str): A TensorFlow Hub feature extraction URL.
      num_classes (int): Number of output neurons/classes in the output layer, should be equal to Number of target classes,
        default 10.

    Returns:
      An uncompiled Keras Sequential model with model_url as feature extractor layer and Dense output layer  with num_classes output neurons. 
    """

    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                             name=f"feature_extractor_layer", 
                                             input_shape=IMAGE_SIZE+(3,), # add 3 color channels to the image size
                                             trainable=False) # freeze the already learned patterns

    # Create our own model
    model = tf.keras.Sequential([
            feature_extractor_layer,
            layers.Dense(num_classes, activation="softmax", dtype='float32', name="output_layer") # ensure output is float32
        ])

    return model

### Creating and testing ResNet TensorFlow Hub Feature Estraction model

In [11]:
# Create Resnet model
resnet_model = create_model(resnet_feature_url,
                             num_classes=train_data_10_percent.num_classes)

In [12]:
# Compile our resnet model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [13]:
# Let's fit our resNet model to the data (10 percent of the food101 dataset)
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data_10_percent,
                                  validation_steps=len(test_data_10_percent),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="resnet50V2_feature_extraction")])

Saving TensorBoard log files to: tensorflow_hub/resnet50V2_feature_extraction/20260115-181008
Epoch 1/5



24/24 [==============================] - 48s 2s/step - loss: 1.9136 - accuracy: 0.3747 - val_loss: 1.0922 - val_accuracy: 0.6504
Epoch 2/5
24/24 [==============================] - 44s 2s/step - loss: 0.8386 - accuracy: 0.7653 - val_loss: 0.7864 - val_accuracy: 0.7416
Epoch 3/5
24/24 [==============================] - 44s 2s/step - loss: 0.5680 - accuracy: 0.8427 - val_loss: 0.6402 - val_accuracy: 0.7988
Epoch 4/5
24/24 [==============================] - 44s 2s/step - loss: 0.4158 - accuracy: 0.9027 - val_loss: 0.5890 - val_accuracy: 0.8124
Epoch 5/5
24/24 [==============================] - 44s 2s/step - loss: 0.3361 - accuracy: 0.9280 - val_loss: 0.5667 - val_accuracy: 0.8220


In [13]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (K  (None, 2048)              23561152  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23581642 (89.96 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23561152 (89.88 MB)
_________________________________________________________________
